In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [5]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V_new = np.zeros(env.nS)
    stopping_condition = np.zeros(env.nS)
    
    while True:
        for state_ind in range(env.nS):
            state_value = 0
            for action_ind in range(env.nA):
                for prob, next_state, reward, done in env.P[state_ind][action_ind]:   
                    state_value += policy[state_ind][action_ind]  * (reward + prob * V[next_state] * discount_factor)
                    
            if abs(state_value - V[state_ind]) < theta:
                stopping_condition[state_ind] = 1
            # Update state value
            V_new[state_ind] = state_value
            
        if np.sum(stopping_condition) == len(stopping_condition):
            break
        else:
            stopping_condition = np.zeros(env.nS)
            
        # Update old state values
        V = V_new[:]
    return np.array(V)

In [10]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [11]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [12]:
v

array([  0.        , -13.99993311, -19.99990384, -21.99989416,
       -13.99993311, -17.99991792, -19.99991088, -19.9999119 ,
       -19.99990384, -19.99991088, -17.9999248 , -13.99994386,
       -21.99989416, -19.9999119 , -13.99994386,   0.        ])